# Introduction to Widgets

## Creating Interactive Elements

### Button

Let us create a basic button. 

In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
button

Button(button_style='primary', description='Click', style=ButtonStyle())

Woww!!!!!!! That's ot. We have created a button with few lines of code. You can play around with the various paramters. 

#### Capturing the button click

We can capture a button click by capturing the click event. Let us give it a try.

In [3]:
def clicked(b):
    print ('You clicked me')
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
button.on_click(clicked)
button

Button(button_style='primary', description='Click', style=ButtonStyle())

Now you can think about what could be achieved with capturing click events

### Drop down (Selection)

Now let us create a drop down 

In [4]:
myDropDown = widgets.Dropdown(
    options=['a', 'b', 'c'],
    value='c',
    description='Choices:',
    disabled=False,
)
myDropDown

Dropdown(description='Choices:', index=2, options=('a', 'b', 'c'), value='c')

Now let us programatically change the value of the drop down

In [5]:
myDropDown.value = 'a'

We can also change the options 

In [6]:
myDropDown.options=[1,2,3,4]

Now let us get more creative and populate our drop down using our own data. For this example we will get all the states from us_states table in nyc database.

### Drop down (Multiple Selection)

In [7]:
multipleFruits = widgets.SelectMultiple(
    options=['apple', 'orange', 'grape','guava','peach'],
    value=['apple'],
    #rows=10,
    description='Fruits',
    disabled=False
)
multipleFruits

SelectMultiple(description='Fruits', index=(0,), options=('apple', 'orange', 'grape', 'guava', 'peach'), value…

In [8]:
import pandas as pd
import psycopg2
import geopandas as gpd
from dateutil import parser

In [9]:
conn = psycopg2.connect('dbname=nyc user=geospatial password=geospatial2023 host=ghhlibrary')

In [10]:
states = pd.read_sql_query(r'select name from us_states',conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_66608\1928515319.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  states = pd.read_sql_query(r'select name from us_states',conn)


In [11]:
states.name.values

array(['Arkansas', 'Arizona', 'Colorado', 'Iowa', 'Illinois', 'Indiana',
       'Kansas', 'Missouri', 'Nebraska', 'New Mexico', 'Nevada',
       'Oklahoma', 'Tennessee', 'Utah', 'West Virginia', 'Idaho',
       'Montana', 'North Dakota', 'South Dakota', 'Vermont', 'Wyoming',
       'Puerto Rico', 'U.S. Virgin Islands', 'Florida', 'Texas',
       'Alabama', 'Georgia', 'Louisiana', 'Mississippi', 'South Carolina',
       'California', 'Connecticut', 'District of Columbia', 'Delaware',
       'Kentucky', 'Massachusetts', 'Maryland', 'North Carolina',
       'New Jersey', 'New York', 'Ohio', 'Pennsylvania', 'Rhode Island',
       'Virginia', 'Wisconsin', 'Michigan', 'Hawaii', 'Maine',
       'Minnesota', 'New Hampshire', 'Oregon', 'Washington', 'Alaska'],
      dtype=object)

In [12]:
statesSelect = widgets.Dropdown(
    options=states.name.values,
    value=states.name.values[0],
    description='States:',
    disabled=False,
)
statesSelect

Dropdown(description='States:', options=('Arkansas', 'Arizona', 'Colorado', 'Iowa', 'Illinois', 'Indiana', 'Ka…

In [13]:
statesSelect.value

'Arkansas'

#### Capturing Selection Event

In [14]:
def select(change):
    print ('The value change from',change['old'],'to',change['new'])
    
statesSelect = widgets.Dropdown(
    options=states.name.values,
    value=states.name.values[0],
    description='States:',
    disabled=False,
)
statesSelect.observe(select, names='value')
statesSelect

Dropdown(description='States:', options=('Arkansas', 'Arizona', 'Colorado', 'Iowa', 'Illinois', 'Indiana', 'Ka…

Now let us spice up things a bit, let us create two drop downs, one for states and one for counties and based on the state select the corresponding counties. For change in state we will query the database, select the counties and update the dropdown. 

In [15]:
def stateSelected(change):
    selectedState = change['new']
    #now let us query all the counties for the state. Unfortunately no state name in counties table, we need a join
    counties = pd.read_sql_query(f"select c.name from us_counties c,us_states s where s.name='{selectedState}' and s.state_fips = c.statefp",conn)
    countiesSelect.options = counties.name.values
    
statesSelect = widgets.Dropdown(
    options=states.name.values,
    value=None,
    description='States:',
    disabled=False,
)
countiesSelect = widgets.Dropdown(
    options=[],
    value=None,
    description='Counties:',
    disabled=False,
)
statesSelect.observe(stateSelected,names='value')
display(statesSelect)
display(countiesSelect)

Dropdown(description='States:', options=('Arkansas', 'Arizona', 'Colorado', 'Iowa', 'Illinois', 'Indiana', 'Ka…

Dropdown(description='Counties:', options=(), value=None)

As you can see you are generating the queries on the fly. 

Another way of adding options is as a list of tuples.

In [16]:
def selected(change):
    print (change['new'])
    
multiple = widgets.Dropdown(
    options=[('One', 1), ('Two', 2), ('Three', 3)],
    value=2,
    description='Number:',
)
multiple.observe(selected,names='value')
multiple

Dropdown(description='Number:', index=1, options=(('One', 1), ('Two', 2), ('Three', 3)), value=2)

We can use this drop down to re-create our previous example in a better way

In [17]:
states = pd.read_sql_query(r'select name,state_fips from us_states',conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_66608\254662904.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  states = pd.read_sql_query(r'select name,state_fips from us_states',conn)


In [18]:
def stateSelected(change):
    selectedState = change['new']
    #now let us query all the counties for the state. Unfortunately no state name in counties table, we need a join
    counties = pd.read_sql_query(f"select c.name from us_counties c where c.statefp='{selectedState}'",conn)
    countiesSelect.options = counties.name.values
    
statesSelect = widgets.Dropdown(
    options=states.values.tolist(),
    value=None,
    description='States:',
    disabled=False,
)
countiesSelect = widgets.Dropdown(
    options=[],
    value=None,
    description='Counties:',
    disabled=False,
)
statesSelect.observe(stateSelected,names='value')
display(statesSelect)
display(countiesSelect)

Dropdown(description='States:', options=(['Arkansas', '05'], ['Arizona', '04'], ['Colorado', '08'], ['Iowa', '…

Dropdown(description='Counties:', options=(), value=None)

Now we don't need to use the join query at all. 

### Text

#### Text Box

In [19]:
textBox = widgets.Text(
    value='',
    placeholder='Type your address here',
    description='Address:',
    disabled=False   
)
textBox

Text(value='', description='Address:', placeholder='Type your address here')

Now let us combine this with a button and create an online geocoder (obviously with the help of geopandas)

In [20]:
def geocode(b):
    results = gpd.tools.geocode([textBox.value])
    print (results)

textBox = widgets.Text(
    value='',
    placeholder='Type your address here',
    description='Address:',
    disabled=False   
)
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
display(textBox)
button.on_click(geocode)
display(button)

Text(value='', description='Address:', placeholder='Type your address here')

Button(button_style='primary', description='Click', style=ButtonStyle())

#### Textarea

In [21]:
textArea = widgets.Textarea(
    value='',
    placeholder='Type your code here',
    description='Code:',
    disabled=False
)
textArea

Textarea(value='', description='Code:', placeholder='Type your code here')

For fun let us execute the program that is written in the text area

In [22]:
def run(b):
    exec(textArea.value)
textArea = widgets.Textarea(
    value='',
    placeholder='Type your code here',
    description='Code:',
    disabled=False
)
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
display(textArea)
button.on_click(run)
display(button)

Textarea(value='', description='Code:', placeholder='Type your code here')

Button(button_style='primary', description='Click', style=ButtonStyle())

#### Label

In [23]:
q = widgets.Label(value="This is my label")
display(q)

Label(value='This is my label')

In [24]:
q.value='Hello'

### Slider

#### Integer Slider

In [25]:
intSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Change:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
intSlider

IntSlider(value=0, continuous_update=False, description='Change:', max=20)

In [26]:
intSlider.value = 10

#### Float Slider

In [27]:
floatSlider = widgets.FloatSlider(
    value=0.3,
    min=0.0,
    max=20.0,
    step=0.01,
    description='Change:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)
floatSlider

FloatSlider(value=0.3, continuous_update=False, description='Change:', max=20.0, step=0.01)

#### Int Range Slider

In [28]:
intRange = widgets.IntRangeSlider(
    value=[5, 7],
    min=0,
    max=10,
    step=1,
    description='Ranges:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
intRange

IntRangeSlider(value=(5, 7), continuous_update=False, description='Ranges:', max=10)

In [29]:
intRange.value

(5, 7)

In [30]:
intRange.value = [4,10]

#### Selection Slider

In [31]:
selSlider = widgets.SelectionSlider(
    options=['2021-01-01','2021-02-01','2021-03-01'],
    value='2021-01-01',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
selSlider

SelectionSlider(continuous_update=False, description='Date:', options=('2021-01-01', '2021-02-01', '2021-03-01…

##### Capturing Slider Events

In [32]:
def square(change):
    print (f'Square of {change["new"]} is {change["new"]**2}')
    
intSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Change:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
intSlider.observe(square,names='value')
intSlider

IntSlider(value=0, continuous_update=False, description='Change:', max=20)

Now let us create something useful. We would want to get the count of Cuyahoga voters between two age ranges. As the Cuyahoga voters is in misc database we need to create a connection first.

In [33]:
conn2 = psycopg2.connect('dbname=misc user=geospatial password=geospatial2023 host=ghhlibrary')

In [34]:
def getTotalForRange(ranges):
    ages = ranges['new']
    query = f'select count(sos_voterid) as total from cuy_voters where age>={ages[0]} and age<{ages[1]}'
    data = pd.read_sql_query(query,conn2)
    total.value = f'There are {data.total.values[0]} voters in this age range'
ageRange = widgets.IntRangeSlider(
    value=[0, 20],
    min=0,
    max=120,
    step=1,
    description='Age Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
total = widgets.Label(value="")
ageRange.observe(getTotalForRange,names='value')
display(ageRange)
display (total)

IntRangeSlider(value=(0, 20), continuous_update=False, description='Age Range:', max=120)

Label(value='')

Another creative example. This time we are going to calculate the total covid positive cases on a monthly basis (from March 2020 to March 2022). 

Inorder to create monthly interval dates between to dates we can get the help of pandas

In [35]:
def getTotalForMonth(changes):
    month = changes['new']
    query = f"select count(mrn) as totalcases from covid_data where test_resul='Positive' and day>='{month}'::date and day<'{month}'::date+interval '1 month'"
    data = pd.read_sql_query(query,conn2)
    total.value = f'There are {data.totalcases.values[0]} covid cases for {month.strftime("%b")} {month.year}'
    
monthly = pd.date_range('2020-03-01','2022-03-01',freq='1MS').date
dateSlider = widgets.SelectionSlider(
    options=monthly,
    value=monthly[0],
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
dateSlider.observe(getTotalForMonth,names='value')
total = widgets.Label(value="")
display(dateSlider)
display(total)

SelectionSlider(continuous_update=False, description='Date:', options=(datetime.date(2020, 3, 1), datetime.dat…

Label(value='')

### Checkbox 

In [36]:
checkBox = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)
checkBox

Checkbox(value=False, description='Check me', indent=False)

In [37]:
checkBox.value = True

#### Listening to Checkbox events

In [38]:
def valueChanged(change):
    print (change['new'])
    
checkBox = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)
checkBox.observe(valueChanged,names='value')
display(checkBox)

Checkbox(value=False, description='Check me', indent=False)

#### RadioButtons

In [39]:
def selectedFruits(change):
    print (change['new'])
fruits = widgets.RadioButtons(
    options=['apple', 'orange', 'pear'],
    value=None,
    layout={'width': 'max-content'},
    description='Select Fruit:',
    disabled=False
)
fruits.observe(selectedFruits,names='value')
fruits

RadioButtons(description='Select Fruit:', layout=Layout(width='max-content'), options=('apple', 'orange', 'pea…

### Date picker

In [40]:
start = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
start

DatePicker(value=None, description='Start Date')

In [41]:
start.value = parser.parse('03/01/2023')

### Listening to Date picker events

In [42]:
def dateChanged(changes):
    print (changes['new'])
start = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
start.observe(dateChanged,names='value')
start

DatePicker(value=None, description='Start Date')

Let us select covid cases between two different dates

In [43]:
def runQuery(b):
    query = f"select count(mrn) as totalcases from covid_data where test_resul='Positive' and day>='{start.value}'::date and day<'{end.value}'::date"
    data = pd.read_sql_query(query,conn2)
    label.value = f'There are {data.totalcases.values[0]} covid cases between {start.value} and {end.value}'
    
start = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end = widgets.DatePicker(
    description='End Date',
    disabled=False
)

button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='Primary'
)
label = widgets.Label(value='')
button.on_click(runQuery)
display(start)
display(end)
display(button)
display(label)

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

Button(button_style='primary', description='Run', style=ButtonStyle())

Label(value='')

One final widget before we get into maps charts and tables

### Animation widget

In [44]:
play = widgets.Play(
    value=50,
    min=0,
    max=100,
    step=1,
    interval=500,   #this is in milliseconds,
    description="Press play",
    disabled=False
)
play

Play(value=50, description='Press play', interval=500)

### Listening to Animation widget

In [45]:
def nextSlide(change):
    print (change['new'])
player = widgets.Play(
    value=50,
    min=0,
    max=100,
    step=1,
    interval=500,   #this is in milliseconds,
    description="Press play",
    disabled=False
)
player.observe(nextSlide,names='value')
player

Play(value=50, description='Press play', interval=500)

In [46]:
conn.close()
conn2.close()